In [1]:
import pandas as pd

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
#Импортируем тексты после лемматизации
data_clienttexts = pd.read_pickle('data_clienttexts_lem.pickle')
data_techtexts = pd.read_pickle('data_techtexts_lem.pickle')

In [96]:
#Импортируем тексты до лемматизации для выбора стандартных ответов суфлеру, выбор по вероятности, числу слов и тд,
#Нужно прописывать на сервер
data_clienttexts_full= pd.read_pickle('data_clienttexts.pickle')
data_techtexts_full = pd.read_pickle('data_textstexts.pickle')

In [101]:
#Переименовываем заголовок текста из необработанного файла, соединяем его с обработанным файлом и ищем лучшие стандартные ответы
data_techtexts_full = data_techtexts_full.rename(columns={'text':'full_text'})
data_clienttexts_full = data_clienttexts_full.rename(columns={'text':'full_text'})
data_techtexts['full_text'] = data_techtexts_full['full_text']
data_clienttexts['full_text'] = data_clienttexts_full['full_text']

In [1]:
#test

In [24]:
#Модели LDA
import pickle
input = open('lda_MC_clients.pickle', 'rb')
lda_MC_clients = pickle.load(input)
input.close()

input = open('lda_MC_tech.pickle', 'rb')
lda_MC_tech = pickle.load(input)
input.close()

In [25]:
#Тексты, словари, корпусы
input = open('texts_clients.pickle', 'rb')
texts_clients = pickle.load(input)
input.close()

input = open('dictionary_clients.pickle', 'rb')
dictionary_clients = pickle.load(input)
input.close()

input = open('corpus_clients.pickle', 'rb')
corpus_clients = pickle.load(input)
input.close()

input = open('texts_tech.pickle', 'rb')
texts_tech = pickle.load(input)
input.close()

input = open('dictionary_tech.pickle', 'rb')
dictionary_tech = pickle.load(input)
input.close()

input = open('corpus_tech.pickle', 'rb')
corpus_tech = pickle.load(input)
input.close()

In [11]:
#Проставляем номера кластеров, имеющих наибольшую вероятность
from operator import itemgetter
data_clienttexts = data_clienttexts.assign(cluster_clients = [sorted(lda_MC_clients[corpus_clients[row]], key = itemgetter(1), reverse=True)[0][0] for row in range(len(data_clienttexts))])

In [21]:
#Проставляем наибольшие вероятности выбранных кластеров
from operator import itemgetter
data_clienttexts = data_clienttexts.assign(cluster_clients_prob = [sorted(lda_MC_clients[corpus_clients[row]], key = itemgetter(1), reverse=True)[0][1] for row in range(len(data_clienttexts))])

In [23]:
data_clienttexts.to_pickle('data_clienttexts_clusters.pickle')

In [92]:
#Проставляем номера кластеров, имеющих наибольшую вероятность
data_techtexts = data_techtexts.assign(cluster_tech
                                       = [sorted(lda_MC_tech[corpus_tech[row]], key = itemgetter(1),
                                                 reverse=True)[0][0] for row in range(len(data_techtexts))])
#Проставляем наибольшие вероятности выбранных кластеров
data_techtexts = data_techtexts.assign(cluster_tech_prob
                                       = [sorted(lda_MC_tech[corpus_tech[row]], key = itemgetter(1),
                                                 reverse=True)[0][1] for row in range(len(data_techtexts))])

In [93]:
data_techtexts.to_pickle('data_techtexts_clusters.pickle')

In [2]:
#Топ-слова модели клиентских текстов
# for topic, top_words in lda_MC_clients.print_topics(num_topics=5, num_words=5):
#     print('Topic', topic, ':', top_words)
#     print('')

In [3]:
#Топ-слова модели текстов тех. поддержки
# for topic, top_words in lda_MC_tech.print_topics(num_topics=5, num_words=5):
#     print('Topic', topic, ':', top_words)
#     print('')

In [207]:
#Остается грамотным образом построить соответствие(оно неоднозначно) для двойной кластеризации
num_topics = 25
topic_words_clients = []
for i in range(num_topics):
    tt = lda_MC_clients.get_topic_terms(i,10)
    topic_words_clients.append([dictionary_clients[pair[0]] for pair in tt])

In [209]:
num_topics = 25
topic_words_tech = []
for i in range(num_topics):
    tt = lda_MC_tech.get_topic_terms(i,10)
    topic_words_tech.append([dictionary_tech[pair[0]] for pair in tt])

In [212]:
for i in range(len(topic_words_clients)):
    topic_words_clients[i] = ' '.join(topic_words_clients[i])

In [214]:
for i in range(len(topic_words_tech)):
    topic_words_tech[i] = ' '.join(topic_words_tech[i])

In [230]:
topic_words_clients_map = [topic_words_clients[i].split(' ') for i in range(len(topic_words_clients))]
corpus_clients_map = [dictionary_tech.doc2bow(text) for text in topic_words_clients_map]

In [231]:
topic_words_tech_map = [topic_words_tech[i].split(' ') for i in range(len(topic_words_tech))]
corpus_tech_map = [dictionary_clients.doc2bow(text) for text in topic_words_tech_map]

In [271]:
#Получаем такое соответствие

In [4]:
#list(lda_MC_clients.get_document_topics(corpus_tech_map, minimum_probability=0.2))

In [5]:
#list(lda_MC_tech.get_document_topics(corpus_clients_map, minimum_probability=0.2))

# Наглядное отображение двух кластеризаций

In [6]:
# for i in range(num_topics):
#     print(i, ' ->', list(lda_MC_clients.get_document_topics(corpus_tech_map, minimum_probability=0.1))[i])

In [7]:
# for i in range(num_topics):
#     print(i, ' ->', list(lda_MC_tech.get_document_topics(corpus_clients_map, minimum_probability=0.1))[i])

In [ ]:
#Далее необходимо подбирать количество кластеров для подтягивания стандартных ответов из FAQ
#или типовых ответов поддержки
#Для этого нужно смотреть перплексию, когерентность моделей,
#отображение есть, посмотрим качественные примеры полученной кластеризации
#https://msk.tele2.ru/help/questions

In [8]:
#data_clienttexts.head()

In [10]:
# faq_first = 'Где можно пополнить счет без комиссии?'
# corpus_faq_first_tech = [dictionary_tech.doc2bow(faq_first.split(' '))]
# corpus_faq_first_clients = [dictionary_clients.doc2bow(faq_first.split(' '))]
# # print(lda_MC_clients[corpus_faq_first_clients[0]])
# # print(lda_MC_tech[corpus_faq_first_tech[0]])
# #(18, 0.52)

In [11]:
#lda_MC_clients.print_topic(18)

In [12]:
#lda_MC_tech.print_topic(8)

In [ ]:
#1----3 18
#Добрый день! как происходит перенос номера?

#Как происходит перенос номера
#Когда и как поменяется оператор? 
#Абонент может выбрать любую дату переноса номера не ранее 8-го дня и не позднее 6-ти месяцев со дня заключения договора об оказании услуг связи с Tele2. Если дата не была определена, перенос осуществляется на 8-й день со дня заключения договора с Tele2. 

#data_techtexts[data_techtexts.cluster_tech == 18].full_text[177]
#data_techtexts[data_techtexts.cluster_tech == 18].full_text[1027]

#2--- 9 18
#Отклонение заявки на перенос номера
#Отклонена заявка на перенос номера в Tele2
#Что делать, если перенос номера был отклонен? 
#Причина отказа в перенесении номера сообщается абоненту в SMS. При отказе по причине несовпадения данных абонента необходимо обратиться к оператору, от которого переносится номер, с письменным заявлением об изменении паспортных данных, если ранее абонент не уведомлял об этом оператора. Баланс переносимого номера должен быть положительным (без задолженности), номер не должен быть заблокирован. После устранения указанных в SMS причин отказа в перенесении номера следует повторно обратиться с заявлением в Tele2.

#3--- 18 8
#Где можно пополнить счет без комиссии?
#Без комиссии счет можно пополнить:
#с помощью банковской карты в разделе «Пополнения и переводы» без дополнительной регистрации;
#с помощью банковской карты в личном кабинете и приложении «Мой Tele2»;
#с помощью услуги Автоплатеж;
#в салонах связи Tele2;
#в салонах связи «Связной» и «Евросеть»;

#4---- 

In [280]:
#Не рискуем выкладывать код на сервер ВК :)
#Приводим примеры удачной кластеризации и код сервера на Flask далее

In [13]:
#Удачное соответствие
faq_first = 'Где можно пополнить счет без комиссии?'
corpus_faq_first_tech = [dictionary_tech.doc2bow(faq_first.split(' '))]
corpus_faq_first_clients = [dictionary_clients.doc2bow(faq_first.split(' '))]
print(lda_MC_clients[corpus_faq_first_clients[0]])
print(lda_MC_tech[corpus_faq_first_tech[0]])
#(18, 0.52)
answer = 'Где можно пополнить счет без комиссии? \
 Без комиссии счет можно пополнить: \
с помощью банковской карты в разделе «Пополнения и переводы» без дополнительной регистрации;\
с помощью банковской карты в личном кабинете и приложении «Мой Tele2»;\
с помощью услуги Автоплатеж;\
в салонах связи Tele2;\
в салонах связи «Связной» и «Евросеть»;'

In [14]:
#Удачное соответствие
faq_first = 'Добрый день! как происходит перенос номера?'
corpus_faq_first_tech = [dictionary_tech.doc2bow(faq_first.split(' '))]
corpus_faq_first_clients = [dictionary_clients.doc2bow(faq_first.split(' '))]
print(lda_MC_clients[corpus_faq_first_clients[0]])
print(lda_MC_tech[corpus_faq_first_tech[0]])

answer = 'Абонент может выбрать любую дату переноса номера не ранее\
8-го дня и не позднее 6-ти месяцев со дня заключения договора об оказании услуг связи\
с Tele2. Если дата не была определена, перенос осуществляется на 8-й день со дня'\

#Стандартные faq могут не решать вопрос
#Для этого мы делали соответствие!
#Выцепляем топ по вероятности из текстов ответов с подходящим кластером
#Пример
#Пренебрегли тут чисткой для примера
print(data_techtexts[data_techtexts.cluster_tech == 18].full_text[177])
print(data_techtexts[data_techtexts.cluster_tech == 18].full_text[1027])

In [15]:
#Удачное соответствие
faq_first = 'Добрый день! как происходит перенос номера?'
corpus_faq_first_tech = [dictionary_tech.doc2bow(faq_first.split(' '))]
corpus_faq_first_clients = [dictionary_clients.doc2bow(faq_first.split(' '))]
print(lda_MC_clients[corpus_faq_first_clients[0]])
print(lda_MC_tech[corpus_faq_first_tech[0]])

answer = 'Где можно пополнить счет без комиссии? \
 Без комиссии счет можно пополнить: \
с помощью банковской карты в разделе «Пополнения и переводы» без дополнительной регистрации;\
с помощью банковской карты в личном кабинете и приложении «Мой Tele2»;\
с помощью услуги Автоплатеж;\
в салонах связи Tele2;\
в салонах связи «Связной» и «Евросеть»;'

In [ ]:
#Дальнейшее:
#Главное дальше: анализ плохих консультантов поддержки и выявление спамящих клиентов, которых можно забанить
#Смотрим по вероятности из базы на сервере, например, через, pypyodbc, если запросов
#за неделю больше 10 по одному кластеру, то считаем спамером
#Если меньше 10, но больше 5 и при этом совпали токены сотрудника, решающего вопрос,
#стоит задуматься о компетентности сотрудника

#Выявление спамящего клиента:
#Поступает запрос, проводим двойную кластеризацию, смотрим историю клиента по данному типу проблемы, если ее много,
#смотреть на возможность спама/плохого консультанта

In [16]:
#data_clienttexts.client_id.value_counts()

In [17]:
#data_clienttexts[data_clienttexts.client_id == 475105].reset_index()[19:22]

In [ ]:
#Пример возможного выброса спама

In [ ]:
#Что не успели
#Не использовать мешок слов gensim, смотреть в сторону word2vec, карт кохонена, bigartm, hdp модели